In [1]:
import geopandas as gpd

In [2]:
dat = gpd.read_file('./data/nhrus/nhru_08_AEA.shp')

In [3]:
def check_val(geom):
    return geom.is_valid

In [4]:
dat['valid'] = dat.geometry.map(check_val)

In [5]:
def fix_geometry(df):
    if df.geometry.is_valid:
        return df.geometry
    
    elif df.geometry.is_valid==False:
        return df.geometry.buffer(0)

In [ ]:
dat['new_geom'] = dat.apply(fix_geometry,axis=1)

In [ ]:
dat['new_valid'] = dat.new_geom.map(check_val)

In [ ]:
dat.new_valid.sum()

In [ ]:
nhrus = dat.hru_id_nat.unique()

In [ ]:
len(nhrus)

In [ ]:
dat = dat.set_geometry('new_geom')

In [ ]:
dat.plot(figsize=(10,10))

In [ ]:
# remove extra columns
del dat['valid']
del dat['new_valid']
del dat['geometry']

In [ ]:
dat.geometry.name

In [15]:
dat.to_file('./data/nhrus/nhru_08_clean.shp')

ValueError: 'bool' is not in list